# Import library


In [33]:
%load_ext autoreload
%autoreload 2

# ===============================
# 1. Import 핵심 라이브러리 (우선순위: numerical/scientific/data)
import numpy as np           # 수치 해석용 (필수)
import pandas as pd          # 데이터프레임 처리용
import math                  # 수학 함수
from tqdm import tqdm        # 진행 상황 시각화
import matplotlib.pyplot as plt           # 플롯
import matplotlib.ticker as ticker        # 플롯 축 설정
import CoolProp.CoolProp as CP            # 열역학/물성치 구할 때 필요

# 2. 커스텀 라이브러리 (src 폴더 내)
import sys
sys.path.append('src')       # 커스텀 모듈 경로 추가

import dhw_ex_model as dem       # 모델 엔진 (src/dhw_ex_model)
import dartwork_mpl as dm        # 플롯 스타일 커스텀

# 3. 플롯 스타일 적용 (dartwork_mpl)
dm.use_style('dmpl_light')   # 다트워크 플롯 스타일 적용 (라이트)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 준비


In [34]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.0),
    'legend': dm.fs(-1.5),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-1.5),
    'cbar_label': dm.fs(-1.5),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-2.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)


# 1. Constant


In [35]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3


# 2. ASHPB2 모델 설정


In [36]:
## 2.1 ASHPB2 모델 객체 생성
ashpb2_model = dem.AirSourceHeatPumpBoiler2(
    # 냉매/사이클/압축기 파라미터
    ref='R410A',
    V_disp_cmp=0.0005,
    eta_cmp_isen=0.7,
    
    # 열교환기 파라미터
    A_ou=20.0,        # 실외기 전열 면적 [m²]
    UA_cond=500.0,    # 응축기 열전달 계수 [W/K]
    UA_evap=500.0,    # 증발기 열전달 계수 [W/K] (기준값)
    A_cross_ou=0.4,   # 실외기 단면적 [m²]
    
    # 실외기 팬 파라미터
    dV_ou_design=2.5,   # 실외기 설계 풍량 [m³/s]
    dP_ou_design=500.0, # 실외기 설계 정압 [Pa]
    eta_motor_ou=0.8,
    eta_fan_ou=0.8,
    
    # 기준 온도
    T0=0.0,    # 기준 외기 온도 [°C]
)


# 3. 테스트 조건 설정


In [37]:
## 3.1 테스트 조건 정의
# 단일 조건 테스트 (예시)
T_tank_w_test = 60.0  # [°C]
T_oa_test = 5.0       # [°C]
Q_cond_load_test = 5000.0  # [W]


# 4. 단일 조건 최적화 테스트


In [39]:
## 4.1 단일 조건에서 최적화 실행
result = ashpb2_model.find_optimal_operation(
    T_tank_w=T_tank_w_test,
    T_oa=T_oa_test,
    Q_cond_load=Q_cond_load_test
)

print("=== 최적화 결과 ===")
print(f"작동 여부: {result.get('is_on', False)}")
if result.get('is_on', False):
    print(f"총 전력 (E_tot): {result.get('E_tot', 0):.2f} [W]")
    print(f"압축기 전력 (E_cmp): {result.get('E_cmp', 0):.2f} [W]")
    print(f"팬 전력 (E_fan_ou): {result.get('E_fan_ou', 0):.2f} [W]")
    print(f"COP: {result.get('cop', 0):.3f}")
    print(f"냉매 유량 (m_dot_ref): {result.get('m_dot_ref', 0):.6f} [kg/s]")
    print(f"응축기 열량 (Q_ref_cond): {result.get('Q_ref_cond', 0):.2f} [W]")
    print(f"증발기 열량 (Q_ref_evap): {result.get('Q_ref_evap', 0):.2f} [W]")
    print(f"최적 팬 풍량 (dV_fan_ou_optimal): {result.get('dV_fan_ou_optimal', 0):.3f} [m³/s]")
    print(f"최적 온도차 (dT_ref_cond): {result.get('dT_ref_cond', 0):.2f} [K]")
    print(f"최적 온도차 (dT_ref_evap): {result.get('dT_ref_evap', 0):.2f} [K]")
    print(f"응축 온도 (T_cond): {result.get('T_cond', 0):.2f} [°C]")
    print(f"증발 온도 (T_evap): {result.get('T_evap', 0):.2f} [°C]")
else:
    print(f"에러: {result.get('error', 'Unknown error')}")


T_air_out_C: -0.022472192813268066
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961920943316727
T_oa: 5.0
T_evap: 0.0
T_cond: 65.0
UA_evap_calc: 500.0
dV_fan_ou: 2.5
T_air_out_C: -0.022472192813268066
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961920943316727
T_oa: 5.0
T_evap: 0.0
T_cond: 65.0
UA_evap_calc: 500.0
dV_fan_ou: 2.5
T_air_out_C: -0.022472192813268066
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961920932345947
T_oa: 5.0
T_evap: 0.0
T_cond: 65.00000001490116
UA_evap_calc: 500.0
dV_fan_ou: 2.5
T_air_out_C: -0.022472192813268066
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961935839703074
T_oa: 5.0
T_evap: -1.4901161193847656e-08
T_cond: 65.0
UA_evap_calc: 500.0
dV_fan_ou: 2.5
T_air_out_C: -0.02247219281315438
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961920943316727
T_oa: 5.0
T_evap: 0.0
T_cond: 65.0
UA_evap_calc: 500.00000238418585
dV_fan_ou: 2.500000014901161
T_air_out_C: -0.022472192813268066
T_oa_C: 5.0
T_ref_evap_avg_C: -0.022961920943316727
T_oa: 5.0
T_evap: 0.0
T_cond: 65.0
UA_evap_calc: 500.0
dV_fan_ou: 2.5

# 5. 파라미터 스윕 테스트


In [20]:
## 5.1 외기 온도 변화에 따른 최적화
T_tank_w_fixed = 60.0  # [°C]
Q_cond_load_fixed = 5000.0  # [W]
T_oa_range = np.arange(-5.0, 20.0, 2.0)  # [°C]

results_sweep = []

for T_oa in tqdm(T_oa_range, desc="외기 온도 스윕"):
    result = ashpb2_model.find_optimal_operation(
        T_tank_w=T_tank_w_fixed,
        T_oa=T_oa,
        Q_cond_load=Q_cond_load_fixed
    )
    
    if result.get('is_on', False):
        results_sweep.append({
            'T_oa': T_oa,
            'T_tank_w': T_tank_w_fixed,
            'Q_cond_load': Q_cond_load_fixed,
            'E_tot': result.get('E_tot', np.nan),
            'E_cmp': result.get('E_cmp', np.nan),
            'E_fan_ou': result.get('E_fan_ou', np.nan),
            'cop': result.get('cop', np.nan),
            'dV_fan_ou_optimal': result.get('dV_fan_ou_optimal', np.nan),
            'dT_ref_cond': result.get('dT_ref_cond', np.nan),
            'dT_ref_evap': result.get('dT_ref_evap', np.nan),
            'T_cond': result.get('T_cond', np.nan),
            'T_evap': result.get('T_evap', np.nan),
            'm_dot_ref': result.get('m_dot_ref', np.nan)
        })
    else:
        results_sweep.append({
            'T_oa': T_oa,
            'T_tank_w': T_tank_w_fixed,
            'Q_cond_load': Q_cond_load_fixed,
            'E_tot': np.nan,
            'E_cmp': np.nan,
            'E_fan_ou': np.nan,
            'cop': np.nan,
            'dV_fan_ou_optimal': np.nan,
            'dT_ref_cond': np.nan,
            'dT_ref_evap': np.nan,
            'T_cond': np.nan,
            'T_evap': np.nan,
            'm_dot_ref': np.nan
        })

df_sweep = pd.DataFrame(results_sweep)
print(f"\n총 {len(df_sweep)}개 조건 테스트 완료")
print(f"성공한 최적화: {len(df_sweep[~df_sweep['E_tot'].isna()])}개")


외기 온도 스윕: 100%|██████████| 13/13 [00:00<00:00, 119.75it/s]


총 13개 조건 테스트 완료
성공한 최적화: 0개


# 6. 시각화

In [9]:
## 6.1 COP vs 외기 온도
X = df_sweep['T_oa'].to_numpy()
Y1 = df_sweep['cop'].to_numpy()

X_LABEL = 'Outdoor air temperature [°C]'
Y1_LABEL = 'COP [-]'

xmin1, xmax1, xint1, xmar1 = -5, 20, 5, 0
ymin1, ymax1, yint1, ymar1 = 0, 6, 1, 0
color_ax1 = 'dm.blue'

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='COP', linewidth=LW[2], color=color_ax1 + '4', marker='o', markersize=3)

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
dm.save_and_show(fig)


In [10]:
## 6.2 전력 vs 외기 온도
X = df_sweep['T_oa'].to_numpy()
Y1 = df_sweep['E_tot'].to_numpy()
Y2 = df_sweep['E_cmp'].to_numpy()
Y3 = df_sweep['E_fan_ou'].to_numpy()

X_LABEL = 'Outdoor air temperature [°C]'
Y1_LABEL = 'Power [W]'

xmin1, xmax1, xint1, xmar1 = -5, 20, 5, 0
ymin1, ymax1, yint1, ymar1 = 0, 3000, 500, 0

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='Total power (E_tot)', linewidth=LW[2], color='dm.blue4', marker='o', markersize=3)
ax1.plot(X, Y2, label='Compressor power (E_cmp)', linewidth=LW[2], color='dm.red4', marker='s', markersize=3)
ax1.plot(X, Y3, label='Fan power (E_fan_ou)', linewidth=LW[2], color='dm.green4', marker='^', markersize=3)

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.legend(fontsize=fs['legend'])

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
dm.save_and_show(fig)


In [11]:
## 6.3 최적 팬 풍량 vs 외기 온도
X = df_sweep['T_oa'].to_numpy()
Y1 = df_sweep['dV_fan_ou_optimal'].to_numpy()

X_LABEL = 'Outdoor air temperature [°C]'
Y1_LABEL = 'Optimal fan flow rate [m³/s]'

xmin1, xmax1, xint1, xmar1 = -5, 20, 5, 0
ymin1, ymax1, yint1, ymar1 = 0, 5, 1, 0
color_ax1 = 'dm.orange'

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='Optimal fan flow rate', linewidth=LW[2], color=color_ax1 + '4', marker='o', markersize=3)

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
dm.save_and_show(fig)


# 7. 결과 저장


In [12]:
## 7.1 결과를 CSV로 저장
df_sweep.to_csv('result/ashpb2_optimization_results.csv', index=False)
print("결과가 'result/ashpb2_optimization_results.csv'에 저장되었습니다.")


결과가 'result/ashpb2_optimization_results.csv'에 저장되었습니다.
